# Basic Secondary Index Query

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

In [5]:
%sh aql -c "set key_send true"

In [6]:
%sh aql -f "../aqlScripts/insert.aql"

In [7]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");
for(int i=0; i<10; i++){
    Key key = new Key("test", "testset", "key"+i);
    System.out.println("Key"+ i + " : "+client.get(null, key));
}

Initialized the client and connected to the cluster.
Key0 : (gen:1),(exp:0),(bins:(name:Sandra),(age:34))
Key1 : (gen:1),(exp:0),(bins:(name:Jack),(age:26))
Key2 : (gen:1),(exp:0),(bins:(name:Jill),(age:20))
Key3 : (gen:1),(exp:0),(bins:(name:James),(age:38))
Key4 : (gen:1),(exp:0),(bins:(name:Jim),(age:46))
Key5 : (gen:1),(exp:0),(bins:(name:Julia),(age:62))
Key6 : (gen:1),(exp:0),(bins:(name:Sally),(age:32))
Key7 : (gen:1),(exp:0),(bins:(name:Sean),(age:24))
Key8 : (gen:1),(exp:0),(bins:(name:Sam),(age:12))
Key9 : (gen:1),(exp:0),(bins:(name:Susan),(age:42))


In [8]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

In [9]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.policy.QueryPolicy;
import com.aerospike.client.query.RecordSet;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setBinNames("name");
stmt.setFilter(Filter.range("age", 38, 46));  //finds inclusive of both 38 and 46
QueryPolicy qp = new QueryPolicy();
RecordSet rs = client.query(qp, stmt);

while(rs.next()){
    Record r = rs.getRecord();
    Key thisKey = rs.getKey();  
    System.out.println(r);
}



(gen:1),(exp:0),(bins:(name:Susan))
(gen:1),(exp:0),(bins:(name:Jim))
(gen:1),(exp:0),(bins:(name:James))


# Cleanup

In [10]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [11]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"